
# 🧹 **End‑to‑End Data Cleaning (Zero‑NaN Guaranteed)**

This notebook demonstrates **how professionals clean real‑world dirty data**.

By the end:
- ✅ No missing values
- ✅ Correct data types
- ✅ Safe for encoding & modeling
- ✅ Interview‑ready explanation

Dataset: `insurance_missing_values`


## 1️⃣ Import Libraries

In [15]:

import pandas as pd
import numpy as np


## 2️⃣ Load Raw Dataset

In [16]:

df = pd.read_csv("insurance_missing_values.csv")
df


,Name,Age,Salary,Gender,Education,Purchased_Insurance
0,Rahul,25,30000,Male,Bachelor,No
1,Anitha,28,42000,Female,Master,Yes
2,NaN,Kumar,55000,Male,Bachelor,Yes
3,Priya,NaN,25000,Female,High School,No
4,Suresh,45,80000,NaN,Master,Yes
5,Meena,32,NaN,Female,Bachelor,Yes
6,Arjun,29,40000,Male,NaN,No
7,Kavya,41,72000,Female,Master,NaN
8,Ramesh,50,"90,000",Male,High School,Yes
9,Divya,twenty six,35000,Female,Bachelor,No


## 3️⃣ Initial Missing Value Audit

In [17]:

df.isna().sum()


Name                   1
Age                    1
Salary                 1
Gender                 1
Education              1
Purchased_Insurance    1
dtype: int64


## 4️⃣ Separate Features & Target

⚠️ **Rule:** Target labels must NEVER be guessed.


In [18]:

X = df.drop("Purchased_Insurance", axis=1)
y = df["Purchased_Insurance"]


## 5️⃣ Drop Rows with Missing Target

In [19]:
mask = y.notna()
X = X[mask]
y = y[mask]

In [29]:
X

,Name,Age,Salary,Gender,Education
0,Rahul,25.0,30000.0,Male,Bachelor
1,Anitha,28.0,42000.0,Female,Master
2,Anitha,30.5,55000.0,Male,Bachelor
3,Priya,30.5,25000.0,Female,High School
4,Suresh,45.0,80000.0,Female,Master
5,Meena,32.0,41000.0,Female,Bachelor
6,Arjun,29.0,40000.0,Male,Bachelor
8,Ramesh,50.0,90000.0,Male,High School
9,Divya,30.5,35000.0,Female,Bachelor


In [ ]:
mask = y.notna()
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

X, y

In [21]:
X

,Name,Age,Salary,Gender,Education
0,Rahul,25,30000,Male,Bachelor
1,Anitha,28,42000,Female,Master
2,NaN,Kumar,55000,Male,Bachelor
3,Priya,NaN,25000,Female,High School
4,Suresh,45,80000,NaN,Master
5,Meena,32,NaN,Female,Bachelor
6,Arjun,29,40000,Male,NaN
8,Ramesh,50,"90,000",Male,High School
9,Divya,twenty six,35000,Female,Bachelor



## 6️⃣ Fix Column Shift Errors & Force Consistent Types

Some rows contain **shifted values** (e.g., name inside Age).
We force everything to string first to stabilize structure.


In [22]:

X = X.astype(str)
X


,Name,Age,Salary,Gender,Education
0,Rahul,25,30000,Male,Bachelor
1,Anitha,28,42000,Female,Master
2,nan,Kumar,55000,Male,Bachelor
3,Priya,nan,25000,Female,High School
4,Suresh,45,80000,nan,Master
5,Meena,32,nan,Female,Bachelor
6,Arjun,29,40000,Male,nan
8,Ramesh,50,"90,000",Male,High School
9,Divya,twenty six,35000,Female,Bachelor



## 7️⃣ Clean Numerical Columns

Problems handled:
- Strings like `"twenty six"`
- Numbers with commas `"90,000"`
- Empty strings


In [23]:

# Remove commas from salary
X["Salary"] = X["Salary"].str.replace(",", "", regex=False)

# Convert Age & Salary safely
X["Age"] = pd.to_numeric(X["Age"], errors="coerce")
X["Salary"] = pd.to_numeric(X["Salary"], errors="coerce")

X[["Age", "Salary"]]


,Age,Salary
0,25.0,30000.0
1,28.0,42000.0
2,NaN,55000.0
3,NaN,25000.0
4,45.0,80000.0
5,32.0,NaN
6,29.0,40000.0
8,50.0,90000.0
9,NaN,35000.0


In [24]:
X

,Name,Age,Salary,Gender,Education
0,Rahul,25.0,30000.0,Male,Bachelor
1,Anitha,28.0,42000.0,Female,Master
2,nan,NaN,55000.0,Male,Bachelor
3,Priya,NaN,25000.0,Female,High School
4,Suresh,45.0,80000.0,nan,Master
5,Meena,32.0,NaN,Female,Bachelor
6,Arjun,29.0,40000.0,Male,nan
8,Ramesh,50.0,90000.0,Male,High School
9,Divya,NaN,35000.0,Female,Bachelor


## 8️⃣ Impute Numerical Missing Values (Median)

In [25]:

for col in ["Age", "Salary"]:
    X[col] = X[col].fillna(X[col].median())

X[["Age", "Salary"]]


,Age,Salary
0,25.0,30000.0
1,28.0,42000.0
2,30.5,55000.0
3,30.5,25000.0
4,45.0,80000.0
5,32.0,41000.0
6,29.0,40000.0
8,50.0,90000.0
9,30.5,35000.0



## 9️⃣ Clean Categorical Columns

Steps:
1. Replace empty strings & 'nan' text
2. Apply mode imputation


In [26]:

categorical_cols = ["Name", "Gender", "Education"]

for col in categorical_cols:
    X[col] = X[col].replace(["nan", "None", ""], np.nan)
    X[col] = X[col].fillna(X[col].mode()[0])

X[categorical_cols]


,Name,Gender,Education
0,Rahul,Male,Bachelor
1,Anitha,Female,Master
2,Anitha,Male,Bachelor
3,Priya,Female,High School
4,Suresh,Female,Master
5,Meena,Female,Bachelor
6,Arjun,Male,Bachelor
8,Ramesh,Male,High School
9,Divya,Female,Bachelor


## 🔟 Final Validation (ZERO NaN CHECK)

In [27]:

X.isna().sum(), X.dtypes


(Name         0
 Age          0
 Salary       0
 Gender       0
 Education    0
 dtype: int64,
 Name          object
 Age          float64
 Salary       float64
 Gender        object
 Education     object
 dtype: object)


## ✅ Final Result

This dataset is now:
- Fully clean
- Type‑safe
- Ready for encoding & ML modeling

💡 **Interview line:**  
> “I validate data cleanliness at every stage and enforce zero‑NaN guarantees before modeling.”


In [28]:
X

,Name,Age,Salary,Gender,Education
0,Rahul,25.0,30000.0,Male,Bachelor
1,Anitha,28.0,42000.0,Female,Master
2,Anitha,30.5,55000.0,Male,Bachelor
3,Priya,30.5,25000.0,Female,High School
4,Suresh,45.0,80000.0,Female,Master
5,Meena,32.0,41000.0,Female,Bachelor
6,Arjun,29.0,40000.0,Male,Bachelor
8,Ramesh,50.0,90000.0,Male,High School
9,Divya,30.5,35000.0,Female,Bachelor
